# Revocation - Bob

## Role: Holder

In this notebook Bob will first be issued a revocable credential from Alice. He will then present it while it is unrevoked. Then Alice will revoke the credential and Bob will attempt to present the revoked credential, which should fail.

## Complete steps 1-6 in the [Alice's revocation notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb)

## 7. Initialise Bob's Controller

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 8. Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    global connection_id
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    global pres_ex_id
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([cred_listener, proof_listener], defaults=True)

Handle Credentials
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: offer_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]


## 9. Continue to [Alice's Notebook](http://127.0.0.1:8888/notebooks/Part%2010%20-%20Revocation.ipynb) to issue a revocable credential.

## 10 Check your credential records

You should have one recording the credential offer from Alice. This should also have shown up as as a print statement from the handler function.

In [3]:
response = await agent_controller.issuer.get_records()
results = response["results"]
print(len(results))
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


1
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]


## 11. Request Credential from Alice

In [4]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: request_sent
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: request_sent
Handle Credentials
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: credential_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]


## 12. Store the Credential

In [5]:
credential_id = "Revocable Credential"
response = await agent_controller.issuer.store_credential(cred_ex_id, credential_id)
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: credential_acked
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'revocation'}, {'name': 'age', 'value': '1337'}]
Credential exchange 93a50281-2722-47ed-b211-7fc9427c0a91, role: holder, state: credential_acked


## 13. Check if the revocation status of the credential

It should not be revoked.

In [6]:
response = await agent_controller.credentials.is_revoked(credential_id)
response

{'revoked': False}

Handle present proof
Role prover, Exchange 2c094bd6-5750-4b43-a44b-7a8937955730 in state request_received


## Continue in Alices Notebook 

Send a presentation request for a non revoked credential proof.


## 15. Fetch Credentials for presentation

Note pres_ex_id is set in the proofs handler.

In [16]:
pres_ex_id
response = await agent_controller.proofs.get_record_by_id(pres_ex_id)
presentation_request = response['presentation_request']
presentation_request

{'name': 'Proof of Personal Information',
 'version': '1.0',
 'requested_attributes': {'0_skill_uuid': {'name': 'skill',
   'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
   'non_revoked': {'to': 1607535945}}},
 'requested_predicates': {},
 'non_revoked': {'to': 1607535946},
 'nonce': '40299591669938516410815'}

In [17]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")
    
if presentation_request != None:
    
    revocation = True

    # include self-attested attributes (not included in credentials)
    credentials_by_reft = {}
    revealed = {}
    self_attested = {}
    predicates = {}

    # select credentials to provide for the proof
    credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
    print(credentials)

    if credentials:
        for row in sorted(
            credentials,
            key=lambda c: dict(c["cred_info"]["attrs"]),
            reverse=True,
        ):
            for referent in row["presentation_referents"]:
                if referent not in credentials_by_reft:
                    credentials_by_reft[referent] = row

    for referent in presentation_request["requested_attributes"]:
        if referent in credentials_by_reft:
            revealed[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ],
                "revealed": True,
            }

        else:
            self_attested[referent] = "South Africa"

    for referent in presentation_request["requested_predicates"]:
        if referent in credentials_by_reft:
            predicates[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ]
            }

    print("\nGenerate the proof")
    proof = {
        "requested_predicates": predicates,
        "requested_attributes": revealed,
        "self_attested_attributes": self_attested,
    }
    print(proof)
    print("\nXXX")
    print("\npredicates:\n{}".format(predicates))
    print("\nrevealed:\n{}".format(revealed))
    print("\nself_attested:\n{}".format(self_attested))
    
else: 
    print("No presenation record identifier")

[{'cred_info': {'referent': 'Revocable Credential', 'attrs': {'name': 'Bob', 'age': '1337', 'skill': 'revocation'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default:CL_ACCUM:60dd0fff-44f9-4971-802a-bf5114cc379b', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1607535945}, 'presentation_referents': ['0_skill_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_skill_uuid': {'cred_id': 'Revocable Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX

predicates:
{}

revealed:
{'0_skill_uuid': {'cred_id': 'Revocable Credential', 'revealed': True}}

self_attested:
{}


## 17. Send Presentation

In [18]:
response = await agent_controller.proofs.send_presentation(pres_ex_id, proof)
print(response)

{'trace': False, 'initiator': 'external', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}], 'non_revoked': {'to': 1607535945}}}, 'requested_predicates': {}, 'non_revoked': {'to': 1607535946}, 'nonce': '40299591669938516410815'}, 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'skill': '19103812416116328689927865197653066156753396496702429306936491003610437194429'}, 'a_prime': '381942011947306989987543054283453326334323694133098820449421479623186912968614231142444133859620201319777096219333992077011317948507089601817019617968802414655682115264224757653563763712926226061261627312479723475302091975946904580943766124443981486985657076246979708666093717270414653136997964353262769912077900072392346272840074808972038576924301101885248131819110087600028864985794612689851132919645477891949193519234878670

Handle present proof
Role prover, Exchange 2c094bd6-5750-4b43-a44b-7a8937955730 in state presentation_sent
Handle present proof
Role prover, Exchange 2c094bd6-5750-4b43-a44b-7a8937955730 in state presentation_acked
